In [42]:
import pyuvsim
import pyuvdata
import pyradiosky
import astropy
import numpy as np
import importlib
importlib.reload(pyuvsim)
from astropy.units import Quantity

In [ ]:
uv = pyuvdata.UVData()
uv.read("/lustre/cosmopipe/2025-06-13/20250613_070132_55MHz.ms")  # Choose an arbitrary data file for reference

# Set antenna positions
antpos = np.zeros_like(uv.telescope.antenna_positions)
antpos[1, 0] = 3000  # Baseline length
antpos_ecef = pyuvdata.utils.ECEF_from_ENU(antpos, center_loc=uv.telescope.location)
telescope_ecef_xyz = Quantity(uv.telescope.location.geocentric).to_value("m")
uv.telescope.antenna_positions  = antpos_ecef - telescope_ecef_xyz

freq_hz = 50e6
channel_width = 2e3
total_time_interval_s = 10
total_freq_interval_hz = 200e3
integration_time = 0.1
time_array_s = np.arange(-total_time_interval_s/2, total_time_interval_s/2, integration_time)
uv.freq_array = np.arange(freq_hz-total_freq_interval_hz/2, freq_hz+total_freq_interval_hz/2, channel_width)
uv.Nfreqs = len(uv.freq_array)
uv.time_array = np.mean(uv.time_array) + time_array_s/86400
uv.Ntimes = len(time_array_s)
uv.Nbls = 1
uv.Nblts = uv.Ntimes
uv.Npols = 1
uv.Nants_data = 2
uv.Nspws = 1
uv.flex_spw_id_array = np.full(uv.Nfreqs, np.mean(uv.spw_array), dtype=int)
uv.ant_1_array = np.zeros(uv.Nblts, dtype=int)
uv.ant_2_array = np.ones(uv.Nblts, dtype=int)
uv.channel_width = np.full(uv.Nfreqs, channel_width)
uv.integration_time = np.full(uv.Nblts, integration_time)
uv.baseline_array = np.full(uv.Nblts, 2048 + 2**16)
uv.data_array = np.zeros((uv.Nblts, uv.Nfreqs, uv.Npols), dtype=complex)
uv.flag_array = np.zeros((uv.Nblts, uv.Nfreqs, uv.Npols), dtype=bool)
uv.nsample_array = np.ones((uv.Nblts, uv.Nfreqs, uv.Npols), dtype=float)
uv.polarization_array = np.array([-5])
uv.set_lsts_from_time_array()
uv.phase_center_app_ra = uv.phase_center_app_ra[:uv.Nblts]
uv.phase_center_app_dec = uv.phase_center_app_dec[:uv.Nblts]
uv.phase_center_frame_pa = uv.phase_center_frame_pa[:uv.Nblts]
uv.phase_center_id_array = uv.phase_center_id_array[:uv.Nblts]
uv.scan_number_array = None
uv.uvw_array = np.zeros((uv.Nblts, 3), dtype=float)
uv.phase_to_time(np.mean(uv.time_array))
# Redefine uvws after phasing
uv.set_uvws_from_antenna_positions()
#uv.uvw_array = np.zeros((uv.Nblts, 3), dtype=float)
#uv.uvw_array[:, 0] = 3e3

beam = pyuvdata.UVBeam()
beam.read("/lustre/rbyrne/LWA_10to100_MROsoil_efields.fits")
beam.select(feeds="e")
beam.peak_normalize()
beam_use_freq = 50e6
use_freq_ind = np.where(beam.freq_array == beam_use_freq)[0]
beam.data_array[:, :, :, :, :] = beam.data_array[:, :, use_freq_ind, :, :]  # Make the beam frequency-invariant
beam_list = pyuvsim.BeamList(beam_list=[beam])

ra_offset_vals_deg = np.linspace(-60, 60, num=11)
dec_offset_vals_deg = np.linspace(-52, 52, num=11)
ra_vals_expanded, dec_vals_expanded = np.meshgrid(ra_offset_vals_deg, dec_offset_vals_deg)
ra_offset_vals_flattened = ra_vals_expanded.flatten()
dec_offset_vals_flattened = dec_vals_expanded.flatten()
source_coords = []
for ind in range(len(ra_offset_vals_flattened)):
    new_coords = astropy.coordinates.SkyCoord([astropy.coordinates.ICRS(
        ra=np.mean(uv.lst_array)*astropy.units.rad + ra_offset_vals_flattened[ind]*astropy.units.deg, 
        dec=uv.telescope.location.lat + dec_offset_vals_flattened[ind]*astropy.units.deg
    )])
    source_coords.append(new_coords)

Setting telescope_location to value in known_telescopes for OVRO-LWA.
The uvw_array does not match the expected values given the antenna positions. The largest discrepancy is 12.140232635605003 meters. This is a fairly common situation but might indicate an error in the antenna positions, the uvws or the phasing.


[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.99998633e+03 -9.01249334e+00  8.76651982e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]


In [78]:
uv.data_array

array([[[0.+0.j],
        [0.+0.j],
        [0.+0.j],
        ...,
        [0.+0.j],
        [0.+0.j],
        [0.+0.j]],

       [[0.+0.j],
        [0.+0.j],
        [0.+0.j],
        ...,
        [0.+0.j],
        [0.+0.j],
        [0.+0.j]],

       [[0.+0.j],
        [0.+0.j],
        [0.+0.j],
        ...,
        [0.+0.j],
        [0.+0.j],
        [0.+0.j]],

       ...,

       [[0.+0.j],
        [0.+0.j],
        [0.+0.j],
        ...,
        [0.+0.j],
        [0.+0.j],
        [0.+0.j]],

       [[0.+0.j],
        [0.+0.j],
        [0.+0.j],
        ...,
        [0.+0.j],
        [0.+0.j],
        [0.+0.j]],

       [[0.+0.j],
        [0.+0.j],
        [0.+0.j],
        ...,
        [0.+0.j],
        [0.+0.j],
        [0.+0.j]]])

In [44]:
zenith_ra = np.mean(uv.lst_array)*astropy.units.rad
print(zenith_ra)
zenith_dec = uv.telescope.location.lat
print(zenith_dec.to(astropy.units.rad))

4.34505325569384 rad
0.649957 rad


In [45]:
print(source_coords[0])

<SkyCoord (ICRS): (ra, dec) in deg
    [(188.95321331, -14.76022273)]>


In [46]:
np.deg2rad(188.95321331)

3.297855704482673

In [47]:
np.deg2rad(-14.76022273)

-0.25761448496620604

In [48]:
ra_offset_vals_flattened[0]

-60.0

In [49]:
dec_offset_vals_flattened[0]

-52.0

In [50]:
decorr_sim = pyuvdata.UVData()
decorr_sim.read("/lustre/rbyrne/decorr_sims/source001.uvfits")

In [51]:
np.shape(decorr_sim.data_array)

(100, 100, 1)

In [52]:
len(decorr_sim.time_array)

100

In [53]:
len(decorr_sim.freq_array)

100

In [54]:
decorr_sim.unproject_phase()

In [55]:
decorr_sim.uvw_array

array([[3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418491e+02, 5.11193276e-01],
       [3.45068195e+01, 6.26418

In [56]:
cat = pyradiosky.SkyModel(
    skycoord = source_coords[ind],
    name = ["source1"],
    spectral_type = "spectral_index",
    spectral_index = [0.],
    reference_frequency = ([np.mean(uv.freq_array)])*astropy.units.Hz,
    stokes = np.array([1,0,0,0]).reshape(4,1,1) * astropy.units.Jy,
)
uv_1freq = uv.select(frequencies = np.min(uv.freq_array), inplace=False)
output_uv = pyuvsim.uvsim.run_uvdata_uvsim(
    input_uv=uv_1freq,
    beam_list=beam_list,
    beam_dict=None,  # Same beam for all ants
    catalog=cat,
    quiet=False,
)

The uvw_array does not match the expected values given the antenna positions. The largest discrepancy is 2963.882518880583 meters. This is a fairly common situation but might indicate an error in the antenna positions, the uvws or the phasing.
The uvw_array does not match the expected values given the antenna positions. The largest discrepancy is 2963.882518880583 meters. This is a fairly common situation but might indicate an error in the antenna positions, the uvws or the phasing.


Nbls: 1
Ntimes: 100
Nfreqs: 1
Nsrcs: 1
Tasks:  100.0
1.00% completed. 0:00:01.758837  elapsed. 0:02:54.124861 remaining. 

2.00% completed. 0:00:02.537499  elapsed. 0:02:04.337460 remaining. 

3.00% completed. 0:00:03.342454  elapsed. 0:01:48.072686 remaining. 

4.00% completed. 0:00:04.149161  elapsed. 0:01:39.579872 remaining. 

5.00% completed. 0:00:05.076004  elapsed. 0:01:36.444077 remaining. 

6.00% completed. 0:00:06.109367  elapsed. 0:01:35.713418 remaining. 

7.00% completed. 0:00:07.117932  elapsed. 0:01:34.566806 remaining. 

8.00% completed. 0:00:07.963627  elapsed. 0:01:31.581709 remaining. 

9.00% completed. 0:00:08.947699  elapsed. 0:01:30.471177 remaining. 

10.00% completed. 0:00:09.904865  elapsed. 0:01:29.143787 remaining. 

11.00% completed. 0:00:10.739686  elapsed. 0:01:26.893825 remaining. 

12.00% completed. 0:00:11.577524  elapsed. 0:01:24.901844 remaining. 

13.00% completed. 0:00:12.421147  elapsed. 0:01:23.126140 remaining. 

14.00% completed. 0:00:13.219332 

The parameter `blt_order` could not be identified for input_uv  so the ordering cannot be restored.The output UVData object will have (time, baseline) ordering.
Recalculating uvw_array without adjusting visibility phases -- this can introduce significant errors if used incorrectly.


In [57]:
output_uv.uvw_array

array([[3.63908090e+01, 6.26311859e+02, 5.21128349e-01],
       [3.63880476e+01, 6.26312020e+02, 5.20927213e-01],
       [3.63852874e+01, 6.26312180e+02, 5.20726173e-01],
       [3.63825261e+01, 6.26312341e+02, 5.20525069e-01],
       [3.63797658e+01, 6.26312501e+02, 5.20324061e-01],
       [3.63770045e+01, 6.26312662e+02, 5.20122989e-01],
       [3.63742443e+01, 6.26312822e+02, 5.19922014e-01],
       [3.63714829e+01, 6.26312983e+02, 5.19720973e-01],
       [3.63687227e+01, 6.26313143e+02, 5.19520030e-01],
       [3.63659613e+01, 6.26313304e+02, 5.19319022e-01],
       [3.63632011e+01, 6.26313464e+02, 5.19118110e-01],
       [3.63604397e+01, 6.26313625e+02, 5.18917134e-01],
       [3.63576795e+01, 6.26313785e+02, 5.18716255e-01],
       [3.63549181e+01, 6.26313946e+02, 5.18515311e-01],
       [3.63521568e+01, 6.26314106e+02, 5.18314383e-01],
       [3.63493965e+01, 6.26314266e+02, 5.18113551e-01],
       [3.63466352e+01, 6.26314427e+02, 5.17912655e-01],
       [3.63438749e+01, 6.26314

In [58]:
uv.uvw_array

array([[3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [30

In [59]:
uv.set_uvws_from_antenna_positions()

In [60]:
uv.uvw_array

array([[3.63908090e+01, 6.26311859e+02, 5.21128349e-01],
       [3.63880476e+01, 6.26312020e+02, 5.20927213e-01],
       [3.63852874e+01, 6.26312180e+02, 5.20726173e-01],
       [3.63825261e+01, 6.26312341e+02, 5.20525069e-01],
       [3.63797658e+01, 6.26312501e+02, 5.20324061e-01],
       [3.63770045e+01, 6.26312662e+02, 5.20122989e-01],
       [3.63742443e+01, 6.26312822e+02, 5.19922014e-01],
       [3.63714829e+01, 6.26312983e+02, 5.19720973e-01],
       [3.63687227e+01, 6.26313143e+02, 5.19520030e-01],
       [3.63659613e+01, 6.26313304e+02, 5.19319022e-01],
       [3.63632011e+01, 6.26313464e+02, 5.19118110e-01],
       [3.63604397e+01, 6.26313625e+02, 5.18917134e-01],
       [3.63576795e+01, 6.26313785e+02, 5.18716255e-01],
       [3.63549181e+01, 6.26313946e+02, 5.18515311e-01],
       [3.63521568e+01, 6.26314106e+02, 5.18314383e-01],
       [3.63493965e+01, 6.26314266e+02, 5.18113551e-01],
       [3.63466352e+01, 6.26314427e+02, 5.17912655e-01],
       [3.63438749e+01, 6.26314

In [61]:
uv.telescope.get_enu_antpos()

array([[-2.63802065e+02,  2.97376191e+02,  8.47599970e-01],
       [-2.29295245e+02,  9.23794682e+02,  1.35879325e+00],
       [-2.96895042e+02,  3.70995527e+02,  1.17227911e+00],
       ...,
       [-7.90131455e+02,  8.49292432e+02,  1.33442206e+00],
       [-9.61875071e+02, -2.54490397e+02, -5.32751862e+00],
       [-1.40646457e+02, -7.08274589e+01, -2.51942914e-01]])

In [62]:
uv.telescope.antenna_positions

array([[-147.36630751,  282.87293142,  237.25708244],
       [  62.43835546,  599.99240352,  736.26444819],
       [-155.52291528,  337.55777362,  296.06265121],
       ...,
       [-452.80273524,  826.03398568,  676.93771516],
       [-918.01427018,  323.85603076, -205.82630477],
       [-144.0703123 ,   29.07086007,  -56.53891113]])

In [63]:
uv.telescope.check()

True

In [64]:
uv.telescope._mount_type

In [65]:
# get antennas positions in ECEF
telescope_ecef_xyz = Quantity(uv.telescope.location.geocentric).to_value("m")
antpos = uv.telescope.antenna_positions + telescope_ecef_xyz

# convert to East, North, Up (ENU) coords.
antpos = pyuvdata.utils.ENU_from_ECEF(antpos, center_loc=uv.telescope.location)

In [66]:
antpos

array([[-2.63802065e+02,  2.97376191e+02,  8.47599970e-01],
       [-2.29295245e+02,  9.23794682e+02,  1.35879325e+00],
       [-2.96895042e+02,  3.70995527e+02,  1.17227911e+00],
       ...,
       [-7.90131455e+02,  8.49292432e+02,  1.33442206e+00],
       [-9.61875071e+02, -2.54490397e+02, -5.32751862e+00],
       [-1.40646457e+02, -7.08274589e+01, -2.51942914e-01]])

In [67]:
np.shape(antpos)

(352, 3)

In [69]:
uv.telescope.antenna_positions

array([[ 9.31322575e-10, -9.31322575e-10,  4.65661287e-10],
       [ 2.64188702e+03, -1.42141936e+03,  4.65661287e-10],
       [ 9.31322575e-10, -9.31322575e-10,  4.65661287e-10],
       ...,
       [ 9.31322575e-10, -9.31322575e-10,  4.65661287e-10],
       [ 9.31322575e-10, -9.31322575e-10,  4.65661287e-10],
       [ 9.31322575e-10, -9.31322575e-10,  4.65661287e-10]])